In [ ]:
### Data Cleaning

In [ ]:
# importing libraries
import pandas as pd
import numpy as np

Reading dataset

In [3]:
data_uni = pd.read_excel("Data Uni Dissertation.xlsx")

data_uni

,Course,UCAS,25 Above,Disability,Ethnicity,Gender,British,English native Language,Parent He attendance,Polar_4_Score,SLC,Care Leaver,Student Visa,Refugee,London Permanent Residence,UCAS Points,English,Maths,A Levels,Btec,Previous work,Bursary,Attendance,AWM year 1,AWM year 2,AWM year 3,Overall AWM,Progression,First Sit,Second Sit,Fails,No Submissions,Late Submission,Pass,Re Takes,desertion
0,BA Business Manangement Enterpreneurship and I...,no,no,no,Asian,Male,no,no,yes,4.0,no,no,yes,no,yes,98.0,5.0,4.0,yes,no,yes,no,86,85,58.0,43.0,62.000000,yes,3,3.0,0,2,2,100.000000,yes,no
1,BA Business Management,no,no,no,White,Male,no,no,yes,2.0,yes,no,no,no,no,101.0,5.0,5.0,yes,no,yes,yes,55,40,32.0,NaN,36.000000,no,1,2.0,5,3,0,83.333333,no,yes
2,BA Business Management Enterpreneurship and In...,no,no,no,Asian,Male,yes,yes,yes,4.0,yes,no,no,no,yes,129.0,4.0,4.0,yes,no,yes,no,57,41,NaN,NaN,41.000000,yes,6,0.0,0,0,0,100.000000,no,yes
3,BA Business Management,no,yes,no,White,Female,no,no,no,3.0,yes,no,no,no,yes,110.0,9.0,8.0,yes,no,yes,no,48,41,43.0,NaN,42.000000,yes,6,0.0,0,0,0,100.000000,no,yes
4,BA Business Management Enterpreneurship and In...,no,no,no,White,Male,yes,yes,yes,4.0,yes,no,no,no,yes,130.0,6.0,5.0,yes,no,yes,no,83,55,49.0,59.0,54.333333,yes,4,2.0,0,2,0,100.000000,no,no
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1028,BA,yes,no,no,Other ethnic background,Female,no,yes,no,1.0,yes,NaN,no,no,no,102.0,4.0,4.0,yes,no,yes,no,55,45,NaN,NaN,45.000000,yes,6,0.0,0,0,1,100.000000,no,yes
1029,BA,yes,no,no,NaN,Male,yes,yes,yes,4.0,yes,no,no,no,yes,109.0,4.0,4.0,yes,no,yes,no,66,77,NaN,NaN,77.000000,yes,6,0.0,0,0,0,100.000000,no,no
1030,BA,no,yes,no,Asian,Female,no,no,no,1.0,no,no,no,no,no,104.0,6.0,5.0,yes,no,yes,no,42,33,NaN,NaN,33.000000,no,1,1.0,2,4,1,33.333333,yes,yes
1031,BA,no,yes,no,Other ethnic background,Male,no,no,yes,4.0,yes,no,no,no,no,101.0,6.0,6.0,no,yes,no,no,60,76,NaN,NaN,76.000000,yes,6,0.0,0,0,0,100.000000,no,no


Exploring Data

In [ ]:
#generating profiling report

from pandas_profiling import ProfileReport
##report = ProfileReport(data_uni)
#report.to_file(output_file= "report.html")

Cleaning Data

In [4]:
# Dealing with  inconsistent values

def trimmer(df):
    column_list = []

    for (columnName, columnData) in df.iteritems():
       
        if columnData.dtype == "object":

            df_replace = columnData.str.strip()

            column_list.append(df_replace)
        else:
            column_list.append(columnData)
            

    return (pd.DataFrame(column_list))

trimmer(data_uni)


,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,...,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023,1024,1025,1026,1027,1028,1029,1030,1031,1032
Course,BA Business Manangement Enterpreneurship and I...,BA Business Management,BA Business Management Enterpreneurship and In...,BA Business Management,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Marketing,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management,Ba Business Management Finance,BA Business Management,BA Business Management Enterpreneurship and In...,BA Business Management Marketing,BA Business Management Enterpreneurship and In...,BA Business Management International Business,Ba Business Management Finance,BA Business Management,BA Business Management,BA Business Management International Business,BA Business Management International Business,Ba Business Management Finance,BA Business Management Enterpreneurship and In...,BA Business Management International Business,BA Business Management,BA Business Management,BA Business Management,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Management Enterpreneurship and In...,BA Business Man

In [11]:
# creating a new df after trimming 
data_uni_df = trimmer(data_uni)
data_uni_df = data_uni_df.transpose( copy = True)

#saving the new data frame
data_uni_df.to_csv("data_uni.csv", index = False)
data_uni_2 = pd.read_csv("data_uni.csv")

imputting missing values

In [12]:
#filling nan with zeros
data_uni_2.iloc[:, [24,25,29]]= data_uni_df.iloc[:, [24,25,29]].fillna(0)

data_uni_2.isna().sum()

Course                          2
UCAS                            0
25 Above                        0
Disability                      0
Ethnicity                      13
Gender                          0
British                        71
English native Language        69
Parent He attendance           37
Polar_4_Score                 118
SLC                             0
Care Leaver                   158
Student Visa                   69
Refugee                         7
London Permanent Residence      5
UCAS Points                    54
English                       160
Maths                         161
A Levels                       60
Btec                          109
Previous work                   3
Bursary                       162
Attendance                      0
AWM year 1                      0
AWM year 2                      0
AWM year 3                      0
Overall AWM                     0
Progression                     0
First Sit                       0
Second Sit    

In [13]:
#imputing missing values with most frequent values
from sklearn.impute import SimpleImputer


imputer = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
imputer.fit(data_uni_2.iloc[:, [0,4,6,7,8,9,11,12,13,14,18,19,20,21]])
data_uni_2.iloc[:, [0,4,6,7,8,9,11,12,13,14,18,19,20,21]]= imputer.transform(data_uni_2.iloc[:, [0,4,6,7,8,9,11,12,13,14,18,19,20,21]])


In [14]:
#imputing missing values with median values
data_num = data_uni_2.select_dtypes(include="number")

imputer_2 = SimpleImputer(missing_values=np.nan, strategy='median')
imputer_2.fit(data_num.iloc[:, [1,2,3]])
data_num.iloc[:, [1,2,3]] = imputer_2.transform(data_num.iloc[:, [1,2,3]])

data_uni_2.iloc[:, [15,16,17]] = data_num.iloc[:, [1,2,3]]

In [ ]:
#Generating report with clean data
#report2 = ProfileReport(data_uni_2)
#report2.to_file(output_file= "report2.html")

Dropping Highly Correlated and unseful Features

In [15]:
data_uni_2= data_uni_2.drop(["Course","AWM year 1", "AWM year 2", "AWM year 3","Overall AWM", "Progression", "Attendance"], axis = 1)

Encoding Feautures

In [16]:
#ethinicty binarisation 

data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("White", "0")
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("Asian", "1")
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("Black/Black British African", "1")
data_uni_2.Ethnicity = data_uni_2.Ethnicity.str.replace("Other ethnic background", "1")

print(data_uni_2.Ethnicity)

0       1
1       0
2       1
3       0
4       0
       ..
1028    1
1029    0
1030    1
1031    1
1032    1
Name: Ethnicity, Length: 1033, dtype: object


In [17]:
# Polar 4 Score Binarisation
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.replace(5, 0)
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.replace(4, 0)
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.replace(3, 0)
data_uni_2.Polar_4_Score = data_uni_2.Polar_4_Score.replace(2, 1)

print(data_uni_2.Polar_4_Score)


0       0.0
1       1.0
2       0.0
3       0.0
4       0.0
       ... 
1028    1.0
1029    0.0
1030    1.0
1031    0.0
1032    0.0
Name: Polar_4_Score, Length: 1033, dtype: float64


In [ ]:
#Correlation Matrix
import matplotlib.pyplot as plt
import seaborn as sns

correlation_matrix = data_uni_2.corr()
plt.figure()
sns.heatmap(correlation_matrix, cmap= "Blues")
correlation_matrix.style.background_gradient(sns.light_palette('blue', as_cmap=True))
plt.show()

In [ ]:
#splitting X and y
y = data_uni_2["desertion"]
X = data_uni_2.drop(["desertion"], axis = 1)

In [ ]:
#Encoding dependent Variable
from sklearn.preprocessing import LabelEncoder
from collections import Counter

le = LabelEncoder()
y_enc = le.fit_transform(y)
Counter(y)

Test 1

In [ ]:
#Encoding independant variables

from sklearn.preprocessing import OneHotEncoder


X_cat = X.select_dtypes(exclude="number")

enc = OneHotEncoder(handle_unknown= "error", sparse = False, drop = "if_binary")

X_cat= np.array(enc.fit_transform(X_cat))

X_cat = pd.DataFrame(X_cat )


X_cat.columns = enc.get_feature_names()
print(X_cat)

In [ ]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_num = X.select_dtypes(include="number")
X_num_colnames= X_num.columns 
a = sc.fit_transform(X_num.to_numpy())
X_num = pd.DataFrame(a)
X_num.columns = X_num_colnames

X_num_cat = pd.concat((X_num, X_cat), axis =1)

print(X_num_cat)

Test 2

In [ ]:
from featurewiz import featurewiz

target = "desertion" 

features, train = featurewiz(data_uni_2, target, corr_limit=0.6, verbose =2, sep=",", header= 0, test_data= 2, feature_engg= "", category_encoders="")

In [ ]:
data_exp = data_uni_2.drop(["25 Above", "Parent He attendance","UCAS Points", "English", "Progression", "Second Sit","No Submissions", "Late Submission", "Pass" ], axis =1)

print (data_exp)

In [ ]:
#from sklearn.compose import ColumnTransformer, make_column_selector
#from sklearn.preprocessing import OneHotEncoder


#ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(sparse= False, drop = "if_binary", handle_unknown= "error"), make_column_selector(dtype_exclude="number"))], remainder='passthrough')

#X_test_2 = np.array(ct.fit_transform(X))
#X_test_2 = pd.DataFrame(X_test_2)

#print(X_test_2)


In [ ]:
#y_test_2 = pd.DataFrame(y_enc)
#y_test_2.rename(columns={0:"desertion"}, inplace= True)
#data_test_2 = pd.concat((X_test_2, y_test_2), axis=1)

#print(data_test_2)

In [19]:
import mlflow
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [20]:
#test1
from pycaret.classification import *

classify = setup(data= data_uni_2, target = "desertion", log_experiment = True, log_plots= True, remove_multicollinearity= True, session_id=2610, multicollinearity_threshold=0.8, log_profile= True, fix_imbalance=True, feature_selection = False, normalize = False,  pca= True, pca_components=0.9, numeric_features=["First Sit", "Fails", "No Submissions", "Late Submission", "Pass",  "Maths", "UCAS Points"])

,Description,Value
0,session_id,2610
1,Target,desertion
2,Target Type,Binary
3,Label Encoded,"no: 0, yes: 1"
4,Original Data,"(1033, 29)"
5,Missing Values,False
6,Numeric Features,9
7,Categorical Features,19
8,Ordinal Features,False
9,High Cardinality Features,False


KeyboardInterrupt: 

In [ ]:
compare_models(include=["rf", "lr", "nb", "ada", "dt","gbc"])

In [ ]:


rf= create_model("rf", cross_validation= True)




In [ ]:

rf= create_model("rf", cross_validation= False)

In [ ]:
tuned_rf= tune_model(rf, optimize = "F1")

In [ ]:
final_rf = calibrate_model(tuned_rf)

In [ ]:
#plot auc
plot_model(final_rf, plot = "auc")

In [ ]:
plot_model(final_rf, plot = "confusion_matrix", plot_kwargs={"percent": True})

In [ ]:
evaluate_model(final_rf)

In [ ]:
gbc = create_model("gbc",cross_validation= False)


In [ ]:
gbc = create_model("gbc",cross_validation= True)

In [ ]:
tuned_gbc= tune_model(gbc, optimize= "F1")

In [ ]:
final_gbc = calibrate_model(tuned_gbc)

In [ ]:
evaluate_model(final_gbc)

In [ ]:
ada = create_model("ada",cross_validation= False)


In [ ]:
ada = create_model("ada",cross_validation= True)

In [ ]:
tune_ada = tune_model(ada, optimize="F1")

In [ ]:
cal_ada = calibrate_model(tune_ada)

In [ ]:
evaluate_model(gbc)

In [ ]:
#test2


classify_2 = setup(data= data_uni_2, target = "desertion", log_experiment = True, log_plots= True, remove_multicollinearity= True, multicollinearity_threshold=0.8, log_profile= True, fix_imbalance=True, feature_selection = True,  session_id= 5970, normalize = False, pca= True, pca_components=0.9, numeric_features=["First Sit", "Fails", "No Submissions", "Late Submission", "Pass",  "Maths", "UCAS Points"])

In [ ]:
compare_models(include=["rf", "ada","gbc","nb","dt","lr"])

In [ ]:
rf_2= create_model("rf", cross_validation= True)

In [ ]:
rf_2= create_model("rf", cross_validation= False)

In [ ]:
tuned_rf_2= tune_model(rf_2, optimize = "F1")

In [ ]:
cal_rf_2 = calibrate_model(tuned_rf_2)

In [ ]:
gbc_2 = create_model("gbc",cross_validation= False)

In [ ]:
gbc_2 = create_model("gbc",cross_validation= True)

In [ ]:
tuned_gbc_2= tune_model(gbc_2, optimize = "F1")

In [ ]:
cal_gbc_2 = calibrate_model(tuned_gbc_2)

In [ ]:
ada_2 = create_model("ada",cross_validation= False)

In [ ]:
ada_2 = create_model("ada",cross_validation= True)

In [ ]:
tuned_ada_2= tune_model(ada_2, optimize = "F1")

In [ ]:
cal_ada_2 = calibrate_model(tuned_ada_2)

In [ ]:
#Test 3
classify_3 = setup(data= data_uni_2, target = "desertion", log_experiment = True, log_plots= True, remove_multicollinearity= True, multicollinearity_threshold=0.8, log_profile= True, fix_imbalance= False, feature_selection = False,  normalize = False, pca= True, pca_components=0.9, numeric_features=["First Sit", "Fails", "No Submissions", "Late Submission", "Pass",  "Maths", "UCAS Points"])

In [ ]:
compare_models(include=["rf", "ada","gbc","nb","dt","lr"])

In [ ]:
rf_3= create_model("rf", cross_validation= True)

In [ ]:
rf_3= create_model("rf", cross_validation= False)

In [ ]:
tuned_rf_3= tune_model(rf_3, optimize = "F1")

In [ ]:
cal_rf_3 = calibrate_model(tuned_rf_3)

In [ ]:
gbc_3 = create_model("gbc",cross_validation= False)

In [ ]:
gbc_3 = create_model("gbc",cross_validation= True)

In [ ]:
tuned_gbc_3= tune_model(gbc_3, optimize = "F1")

In [ ]:
cal_gbc_2 = calibrate_model(tuned_gbc_2)

In [ ]:
ada_3 = create_model("ada",cross_validation= False)

In [ ]:
ada_3 = create_model("ada",cross_validation= True)

In [ ]:
tuned_ada_3= tune_model(ada_3, optimize = "F1")

In [ ]:
cal_ada_3 = calibrate_model(tuned_ada_3)

In [ ]:
classify_4 = setup(data= data_uni_2, target = "desertion", log_experiment = True, log_plots= True, remove_multicollinearity= True, multicollinearity_threshold=0.8, log_profile= True, fix_imbalance=True, feature_selection = True,  session_id= 5970, normalize = False, pca= False, pca_components=0.9, numeric_features=["First Sit", "Fails", "No Submissions", "Late Submission", "Pass",  "Maths", "UCAS Points"])

In [ ]:
compare_models(include=["rf", "ada","gbc","nb","dt","lr"])

In [ ]:
gbc_4=create_model("gbc")

In [361]:
evaluate_model(gbc_4)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Hyperparameters', 'param…